In [1]:
from data import load_beeline
from logger import LightLogger, load_logger
from runner import runGRNVAE
from evaluate import extract_edges

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
configs = {    
    'train_split': 0.8,
    'train_split_seed': None, 
    'batch_size': 64,

    'hidden_dim': 128,
    'z_dim': 1,
    'train_on_non_zero': True,
    'dropout_augmentation': 0.1,
    'cuda': True,
    
    'alpha': 100,
    'beta': 1,
    'delays_on_sparse': 30,
    
    'n_epochs': 250,
    'eval_on_n_steps': 10,
    'lr_nn': 1e-4,
    'lr_adj': 2e-5,
    'K1': 1,
    'K2': 1
}

In [3]:
# Load data from a BEELINE benchmark
data, ground_truth = load_beeline(
    data_dir='data', 
    benchmark_data='mDC', 
    benchmark_setting='500_Non-ChIP'
)

logger = LightLogger()
# runGRNVAE initializes and trains a GRNVAE model with the configs specified. 
vae = runGRNVAE(
    data.X, configs, ground_truth=ground_truth, logger=logger)

100%|██████████| 250/250 [00:15<00:00, 16.54it/s]


In [4]:
logs = logger.to_df()
predicted_A = vae.get_adj()

gene_names = data.var_names
predicted_edges = extract_edges(A=predicted_A, gene_names=gene_names)
predicted_edges.head(10)

,Source,Target,EdgeWeight,AbsEdgeWeight
0,H2-EB1,H2-AB1,0.013215,0.013215
1,H2-AB1,H2-EB1,0.013200,0.013200
2,H2-EB1,H2-AA,0.013155,0.013155
3,H2-AA,H2-AB1,0.013095,0.013095
4,H2-AB1,H2-AA,0.013028,0.013028
5,H2-AA,H2-EB1,0.012872,0.012872
6,H2-EB1,CD74,0.012855,0.012855
7,H2-AB1,CD74,0.012717,0.012717
8,CD74,H2-AA,0.012612,0.012612
9,H2-AA,CD74,0.012606,0.012606
